In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib widget
import matplotlib.pyplot as plt
from scipy.constants import R
import pandas as pd
import numpy as np
import os
import glob 
import psycopg2
import sqlalchemy as sq
from sqlalchemy.orm import Session

import ipywidgets as widgets

import pygaps as pg
from bet_class import BET
from bet_db import BetdbExporter
from tomocatdb.data_model import *

# BET analysis dashboard - BelMiniX

Written in Python using [pyGAPS](https://pygaps.readthedocs.io/en/master/)
, [Voila](https://voila.readthedocs.io/en/stable/using.html) and JupyterLab.  

Source code available on: [https://github.uio.no/SMN-Catalysis/Gas-Adsorption-Analysis](https://github.uio.no/SMN-Catalysis/Gas-Adsorption-Analysis)

by,  
Nicolai Haaber Junge  
Heterogeneous Catalysis,  
Centre for materials science and nanotechnology  
University of Oslo  
11.07.2021

### This dashboard performs BET analysis on BelMiniX .DAT measurment files.

The BET analysis uses the Rouquerol rules for selecting a pressure range. Note,
that you can also set a manual pressure range if you wish.

With this dashboard you can:
- Do BET analysis in single and batch mode
- Quickly batch process a folder of .DAT files
- Export results as individual or multiple files in excel and csv format.

---

In [ ]:
##### WIDGETS ######## 

"""
In this cell all widgets are created and defined for the GUI-program
"""

## INPUT 
lbl = widgets.Label('Select the folder "above" your measurment folder', layout=widgets.Layout(grid_area='lbl', justify_self='start'))
main_folder_lbl = widgets.Label('Main folder', grid_area='main_f_lbl')
main_folder = widgets.Text(layout=widgets.Layout(width='98%', height='30px', grid_area='main_f'))  #  Main folder select

select_folder_lbl = widgets.Label('Selected folder', layout=widgets.Layout(grid_area='sel_f_lbl'))
select_folder = widgets.Dropdown(layout=widgets.Layout(width='98%', height='30px', grid_area='sel_f'))

file_lbl = widgets.Label('File select (ctrl + click to select multiple)', layout=widgets.Layout(grid_area='file_lbl'))
file_folder = widgets.SelectMultiple(layout=widgets.Layout(width='98%', height='325px', grid_area='file_f'))

## EXPORT

def create_toggle(descr, area_name, disabled=False):
    return widgets.ToggleButton(description=descr, disabled=disabled,
                 layout=widgets.Layout(width='auto', grid_area=area_name, justify_self='start'))

check1 = create_toggle('Export folder', 'check1')
check2 = create_toggle('Export selected', 'check2')
json_but = create_toggle('To json', 'json_check', disabled=True)
excel_bu = create_toggle('To excel', 'excel_check', disabled=True)
csv_but = create_toggle('To csv', 'csv_check', disabled=True)
db_but = create_toggle('To db', 'db_check', disabled=True)

output_folder_lbl = widgets.Label('Output directory (default to input file location)',layout=widgets.Layout(width='auto', grid_area='output_f_lbl'))
output_folder = widgets.Text(layout=widgets.Layout(width='auto', grid_area='output_fold'))


to_filetype_buttons = [json_but, excel_bu, csv_but]

batch_proc_button = widgets.Button(description='Export', disabled=True, layout=widgets.Layout(width='40%', area_grid='bath_p_button', justify_self='center'))

label1 = widgets.Label('Batch export options', layout=widgets.Layout(width='auto', grid_area='label1'))
label2 = widgets.Label('File options', layout=widgets.Layout(width='auto', grid_area='label2'))


note = 'It is recommended to set a manual BET \npressure range when batch exporting.\n\n'\
        'Run a single BET calculation first to \nget an idea of a good pressure range.'

label_opt = widgets.Textarea(note, disabled=True, layout=widgets.Layout(width='auto', height='90px', grid_area='label_opt'))

## SQL TAB

note_sql_text = 'To commit an analysis to a database server the server credentials are needed.\n' \
                'Currently, only Postgres is supported\n\nThis feature requires the tomocatdb package\n' 

note_sql = widgets.Textarea(note_sql_text, disabled=True, layout=widgets.Layout(width='auto', height='140px', grid_area='note_sql'))

user_name = widgets.Text(placeholder='Server Username', layout=widgets.Layout(width='auto', grid_area='user-name'))
password = widgets.Text(placeholder='Server Password', layout=widgets.Layout(width='auto', grid_area='password'))
host = widgets.Text(placeholder='Host Adress', layout=widgets.Layout(width='auto', grid_area='host'))
database = widgets.Text(placeholder='Database Name', layout=widgets.Layout(width='auto', grid_area='database'))

establish_conn = widgets.Button(description='Connect', layout=widgets.Layout(width='auto', area_grid='conn'))
conn_check = widgets.Valid(value=False, description='Connected', indent=False, layout=widgets.Layout(width='30px', area_grid='conn_check'))

## BET TAB

isotherm_out = widgets.Output()
bet_plot = widgets.Output()
roq_plot = widgets.Output()
bet_res = widgets.Output(layout=widgets.Layout(height='50%', area_grid='res_bet', justify_item='center'))

plot_outputs = [isotherm_out, bet_plot, roq_plot, bet_res]

run_analysis = widgets.Button(description='Calc. BET', disabled=True, layout=widgets.Layout(grid_area='run_anal'))
manual_press_check = widgets.Checkbox(False, description='Set manual pressure range', indent=False, layout=widgets.Layout(width='auto', grid_area='man_pres_check', justify_item='start'))

press_range_slider = widgets.FloatRangeSlider(
    value=[1e-3, 1e-1],
    min=0,
    max=0.1,
    step=1e-3,
    disabled=True,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
    layout=widgets.Layout(width='auto', grid_area='pres_range')
)

material  = widgets.Label('Material (optional)', layout=widgets.Layout(width='auto', grid_area='mat'))
material_name = widgets.Text(layout=widgets.Layout(width='auto', grid_area='mat_name'))
meas_id = widgets.Label('Measurmet ID. (optional)', layout=widgets.Layout(width='auto', grid_area='meas_id'))
meas_id_name  = widgets.Text(layout=widgets.Layout(width='auto', grid_area='meas_id_name'))


In [ ]:
###### EVENTS #####

"""
In this cell all widget events are defined.
"""

## INPUT
def on_main_folder_change(change):
    folder = os.listdir(change.new)
    select_folder.options = folder  # The widget event can change the state of other widgets like so.


def on_selected_folder_change(change):
    fold = os.path.join(main_folder.value, change.new)
    files = [_ for _ in os.listdir(fold) if _.endswith('.DAT')]
    file_folder.options = files
    
def on_select_file_change(change):
    if len(file_folder.value) > 1:
        run_analysis.disabled = True
    else:
        run_analysis.disabled = False
    
main_folder.observe(on_main_folder_change, names='value')  # WHen the widget change value we call the function "on_main_folder_change"
select_folder.observe(on_selected_folder_change, names='value')  
file_folder.observe(on_select_file_change, names='value')

## EXPORT 
def export_folder_click(change):
    if check2.value:
        check2.value = change.old
    for b in to_filetype_buttons:
        b.disabled = change.old
        
    if conn_check.value:
        db_but.value = change.old
        db_but.disabled = change.old
    
    if change.new == False and check2.value == False:
        for b in to_filetype_buttons:
            b.value = False
            b.disabled = True
        
        db_but.value = False
        db_but.disabled = True
        
        batch_proc_button.disabled = True
    
def export_selected_click(change):
    if check1.value:
        check1.value = change.old
        
    for b in to_filetype_buttons:
        b.disabled = change.old
        
    if conn_check.value:
        db_but.value = change.old
        db_but.disabled = change.old
        
    if change.new == False and check1.value == False:
        for b in to_filetype_buttons:
            b.value = False
            b.disabled = True
            
        db_but.value = False
        db_but.disabled = True
            
        batch_proc_button.disabled = True    
        
check1.observe(export_folder_click, names='value')
check2.observe(export_selected_click, names='value')

filetype_buttons = [json_but, excel_bu, csv_but, db_but]
def on_file_type_click(change):
    for bt in filetype_buttons:
        if bt.value:
            batch_proc_button.disabled = False
            break
        batch_proc_button.disabled = True
    
json_but.observe(on_file_type_click, names='value')
excel_bu.observe(on_file_type_click, names='value')
csv_but.observe(on_file_type_click, names='value')
db_but.observe(on_file_type_click, names='value')

#### batch process clicked

def if_folder_selected(curr_folder, limits):
    all_files = [_ for _ in os.listdir(curr_folder) if _.endswith('.DAT')]
    
    if os.path.isdir(output_folder.value):
        result_path = output_folder.value
    else:
        result_path = None
        
    for file in all_files:
        file_path = os.path.join(curr_folder, file)
        bet = BET(file_path, limits=limits)
        
        if json_but.value:
            bet.to_json()
        if excel_bu.value:
            bet.to_excel(filepath=result_path)
        if csv_but.value:
            bet.to_csv()
        if db_but.value:
            
            username = user_name.value
            passw = password.value
            host_name = host.value
            db_name = database.value
            
            conn_string = 'postgresql+psycopg2://{}:{}@{}/{}'.format(
                                                                username,
                                                                passw,
                                                                host_name,
                                                                db_name
                                                        )
            bet_exp = BetdbExporter(conn_string , bet)
            bet_exp.push_to_db()
    

def if_selected_files(curr_folder, limits):
    files = file_folder.value
    
    if os.path.isdir(output_folder.value):
        result_path = output_folder.value
    else:
        result_path = None
        
    for file in files:
        file_path = os.path.join(curr_folder, file)
            
        bet = BET(file_path, limits=limits)
        
        if json_but.value:
            bet.to_json()
        if excel_bu.value:
            bet.to_excel(filepath=result_path)
        if csv_but.value:
            bet.to_csv()
        if db_but.value:
            
            username = user_name.value
            passw = password.value
            host_name = host.value
            db_name = database.value
            
            conn_string = 'postgresql+psycopg2://{}:{}@{}/{}'.format(
                                                                username,
                                                                passw,
                                                                host_name,
                                                                db_name
                                                        )
            bet_exp = BetdbExporter(conn_string , bet)
            bet_exp.push_to_db()
        
def on_batch_process_click(b):
    curr_folder = os.path.join(main_folder.value, select_folder.value)
    
    limits = None
    if manual_press_check.value:
        limits = press_range_slider.value
    
    if check1.value:
        if_folder_selected(curr_folder, limits)
    else:
        if_selected_files(curr_folder, limits)
    
        
batch_proc_button.on_click(on_batch_process_click) 

## SQL-Database

def postgres_test(conn_string):
    try:
        conn = psycopg2.connect(conn_string)
        conn.close()
        return True
    except:
        return False

def on_sql_conn_click(b):
    conn_string = "dbname={} user={} password={} host={}".format(
                database.value,
                user_name.value,
                password.value,
                host.value
            )
    if postgres_test(conn_string):
        conn_check.value = True
    else:
        conn_check.value = False

establish_conn.on_click(on_sql_conn_click)

## BET
def run_bet(b):
    for output in plot_outputs:
        output.clear_output()
    
    file_select = os.path.join(main_folder.value, select_folder.value, file_folder.value[0])
    
    limits = None
    if manual_press_check.value:
        limits = press_range_slider.value
    
    bet = BET(file_select, limits=limits)
    with bet_res:
        res = bet.BET_results 
        df = pd.DataFrame(res.values(), columns=['value'])
        df = df.set_index(pd.Index([*res]))
        df['units'] = ['m^2/g', '', 'mol/g', '', '', '', '', '', '', '', '', '']
        display(df)
        
    with isotherm_out:
        bet.plot_isotherm()
    
    with bet_plot:
        bet.plot_bet()

    with roq_plot:
        bet.plot_roq()
        
def manual_press(change):
    press_range_slider.disabled = change.old

run_analysis.on_click(run_bet)
manual_press_check.observe(manual_press, names='value')

In [ ]:
######### LAYOUT ########

## INPUT
input_options = widgets.GridBox(
        children=[lbl, main_folder_lbl, main_folder, 
                  select_folder_lbl, select_folder, 
                  file_lbl, file_folder],
        layout=widgets.Layout(
            width='100%',
            grid_template_rows='auto auto auto auto auto auto',
            grid_template_columns='25% 25% 25% 25%',
            grid_template_areas='''
            "lbl lbl lbl lbl"
            "main_f_lbl main_f_lbl main_f_lbl main_f_lbl"
            "main_f main_f main_f main_f"
            "sel_f_lbl sel_f_lbl sel_f_lbl sel_f_lbl"
            "sel_f sel_f sel_f sel_f"
            "file_lbl file_lbl file_lbl file_lbl"
            "file_f file_f file_f file_f"
            ''')
       )

## EXPORT
export_options = widgets.GridBox(children=[label1, check1, check2, output_folder_lbl, output_folder, label_opt, 
                                           label2, json_but, excel_bu, csv_but, db_but],
        layout=widgets.Layout(
            width='100%',
            grid_template_rows='auto auto auto auto 50px auto 70px',
            grid_template_columns='25% 25% 25% 25%',
            grid_template_areas='''
            "label1 label1 label1 label1"
            "check1 check1 check2 check2"
            "label_opt label_opt label_opt label_opt"
            "output_f_lbl output_f_lbl output_f_lbl output_f_lbl"
            "output_fold output_fold output_fold output_fold"
            "label2 label2 label2 label2"
            "json_check excel_check csv_check db_check"
            ''')
       )

## SQL TAB
sql_options = widgets.GridBox(
        children=[note_sql, user_name, password, host, database, establish_conn, conn_check],
        layout=widgets.Layout(
            width='100%',
            grid_template_rows='160px auto 50px 50px',
            grid_template_columns='25% 25% 25% 25%',
            grid_template_areas='''
            "note_sql note_sql note_sql note_sql"
            "user-name user-name password password"
            "host host database database"
            "conn conn conn_check conn_check"
            ''')
       )

### JOIN input, export, sql

layout_style = widgets.Layout(flex_flow='column',
                    align_items='center')

tab_contents = ['Data Input', 'Batch Export', 'SQL Database']
children = [input_options, widgets.VBox([export_options, batch_proc_button], layout=layout_style), sql_options]
tab = widgets.Tab(layout=widgets.Layout(width='99%', height='600px', grid_area='tab'))
tab.children = children
it = [tab.set_title(i, title) for i, title in enumerate(tab_contents)]

## BET

BET_main = widgets.GridBox(
    children=[run_analysis, manual_press_check, press_range_slider],
        layout=widgets.Layout(
            width='100%',
            grid_template_rows='auto 50px 60px',
            grid_template_columns='25% 25% 25% 25%',
            grid_template_areas='''
            "man_pres_check man_pres_check man_pres_check man_pres_check"
            "pres_range pres_range pres_range pres_range"
            ". run_anal run_anal ."
            ''')
       )


BET_options = widgets.GridBox(
    children=[material, material_name, meas_id, meas_id_name],
        layout=widgets.Layout(
            width='100%',
            grid_template_rows='auto auto auto',
            grid_template_columns='25% 25% 25% 25%',
            grid_template_areas='''
            "mat mat mat mat"
            "mat_name mat_name mat_name mat_name"
            "meas_id meas_id meas_id meas_id"
            "meas_id_name meas_id_name meas_id_name meas_id_name"
            ''')
       )

box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',)

tab_BET_contents = ['BET', 'Options']
BET_children = [widgets.VBox(children=[BET_main, bet_res], layout=box_layout), BET_options]
tab_BET = widgets.Tab(layout=widgets.Layout(width='99%', height='600px', grid_area='bet_tab'))
tab_BET.children = BET_children
bet_titles = [tab_BET.set_title(i, title) for i, title in enumerate(tab_BET_contents)]

## PLOTTING
tab_plot_contents = ['BET Plot', 'Isotherm', 'Rouquerol Plot']
plot_children = [bet_plot, isotherm_out, roq_plot]
tab_plot = widgets.Tab(layout=widgets.Layout(width='95%', height='600px', grid_area='plot_tab'))
tab_plot.children = plot_children
plot_titles = [tab_plot.set_title(i, title) for i, title in enumerate(tab_plot_contents)]


In [ ]:
# MAIN WINDOW

### Layout
main = widgets.GridBox(children=[tab, tab_BET, tab_plot],
        layout=widgets.Layout(
            justify_content='center',
            width='99%',
            grid_template_rows='auto',
            grid_template_columns='25% 22% 50%',
            grid_template_areas='''
            "tab bet_tab plot_tab"
            ''')
       )

display(main)

In [ ]:
res